In [ ]:
import numpy as np
import pandas as pd
import optuna
import warnings
import itertools
import time
import matplotlib.pyplot as plt
import seaborn as sns

# Dataset e Preprocessing
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectFromModel
from sklearn.base import BaseEstimator, RegressorMixin

# Modelli
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from catboost import CatBoostRegressor

# Configurazione
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings('ignore')

# ---------------------------------------------------------
# 1. CARICAMENTO E PULIZIA DATI
# ---------------------------------------------------------
print("1. Caricamento e Pulizia Dati...")
start_global = time.time()

data = fetch_california_housing()

X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name="MedHouseVal")
df_full = pd.concat([X, y], axis=1)
df = df_full.sample(frac=0.10, random_state=42)

# Rimuoviamo il cap a 5.0
df = df[df['MedHouseVal'] < 5.0]

def remove_outliers_iqr(df, columns):
    df_clean = df.copy()
    indices_to_drop = []
    for col in columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df_clean[(df_clean[col] < Q1 - 2.0*IQR) | (df_clean[col] > Q3 + 2.0*IQR)].index
        indices_to_drop.extend(outliers)
    return df_clean.drop(list(set(indices_to_drop)))

cols_clean = ['AveRooms', 'AveBedrms', 'AveOccup', 'MedInc']
df = remove_outliers_iqr(df, cols_clean)
print(f"   -> Dataset pulito: {df.shape}")

# ---------------------------------------------------------
# 2. FEATURE ENGINEERING MASSIVA
# ---------------------------------------------------------
print("2. Generazione Feature 'Combo'...")

def generate_comprehensive_features(df_input, cols_to_combine):
    df_eng = df_input.copy()
    math_cols = [c for c in cols_to_combine if c not in ['Latitude', 'Longitude', 'Geo_Cluster']]
    
    # Trucco Rotazione Coordinate per Alberi
    df_eng['Rot_45_LatLon'] = df_eng['Latitude'] + df_eng['Longitude']
    df_eng['Rot_N45_LatLon'] = df_eng['Latitude'] - df_eng['Longitude']

    # A. LOGARITMI
    for col in math_cols:
        if df_eng[col].min() >= 0:
            df_eng[f'LOG_{col}'] = np.log1p(df_eng[col])

    # B. MOLTIPLICAZIONI
    for col1, col2 in itertools.combinations(math_cols, 2):
        col_name = f'MULT_{col1}_x_{col2}'
        df_eng[col_name] = df_eng[col1] * df_eng[col2]

    # C. DIVISIONI
    for col1, col2 in itertools.permutations(math_cols, 2):
        col_name = f'RATIO_{col1}_div_{col2}'
        df_eng[col_name] = df_eng[col1] / (df_eng[col2] + 1e-5)

    return df_eng

# Geo Features Base
sf_coords = (37.7749, -122.4194)
la_coords = (34.0522, -118.2437)
df['Dist_SF'] = np.sqrt((df['Latitude'] - sf_coords[0])**2 + (df['Longitude'] - sf_coords[1])**2)
df['Dist_LA'] = np.sqrt((df['Latitude'] - la_coords[0])**2 + (df['Longitude'] - la_coords[1])**2)

coords = df[['Latitude', 'Longitude']]
kmeans = KMeans(n_clusters=15, random_state=42, n_init=10)
df['Geo_Cluster'] = kmeans.fit_predict(StandardScaler().fit_transform(coords))

X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']

cols_for_math = [c for c in X.columns if c != 'Geo_Cluster']
X_full = generate_comprehensive_features(X, cols_for_math)
X_full.replace([np.inf, -np.inf], np.nan, inplace=True)
X_full.fillna(0, inplace=True)
print(f"   -> Totale Feature Generate: {X_full.shape[1]}")

# ---------------------------------------------------------
# 3. SELEZIONE FEATURE CON XGBOOST GPU
# ---------------------------------------------------------
print("3. Selezione Feature tramite GPU...")
scaler = RobustScaler()
# Nota: fit non richiede scaling per alberi, ma utile averlo per prassi
X_scaled = scaler.fit_transform(X_full)

selector_model = XGBRegressor(
    n_estimators=500, max_depth=8, learning_rate=0.05,
    tree_method='hist', device='gpu', n_jobs=1, random_state=42
)
selector_model.fit(X_full, y)
selection = SelectFromModel(selector_model, prefit=True, threshold='1.25*median')

X_selected = X_full.loc[:, selection.get_support()]
print(f"   -> Feature Sopravvissute: {X_selected.shape[1]}")

# Split Principale (Train / Test)
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# ---------------------------------------------------------
# 4. OPTUNA REALE CON EARLY STOPPING CORRETTO
# ---------------------------------------------------------
print("\n4. Tuning Iperparametri con Optuna e Early Stopping...")

# Creiamo un set di validazione SOLO per Optuna per monitorare l'early stopping
X_opt_train, X_opt_val, y_opt_train, y_opt_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

N_TRIALS = 20  # Numero di trial (aumentare per risultati migliori)

# --- A. XGBoost Optimization ---
def objective_xgb(trial):
    params = {
        'n_estimators': 4000,  # Alto, tanto ferma l'early stopping
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 0.95),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.95),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 10.0, log=True),
        'tree_method': 'hist',
        'device': 'gpu',
        'n_jobs': 1,
        'random_state': 42,
        # Early Stopping nel costruttore per versioni recenti (o gestito in fit)
        'early_stopping_rounds': 100
    }
    
    model = XGBRegressor(**params)
    
    # Verbose=False sopprime i log, eval_set serve per l'early stopping
    model.fit(
        X_opt_train, y_opt_train,
        eval_set=[(X_opt_val, y_opt_val)],
        verbose=False
    )
    
    preds = model.predict(X_opt_val)
    return np.sqrt(mean_squared_error(y_opt_val, preds))

print("   -> Optimizing XGBoost...")
study_xgb = optuna.create_study(direction='minimize')
study_xgb.optimize(objective_xgb, n_trials=N_TRIALS)

# --- B. CatBoost Optimization ---
def objective_cat(trial):
    params = {
        'iterations': 4000,
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'depth': trial.suggest_int('depth', 6, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'bootstrap_type': 'Bernoulli',
        'subsample': trial.suggest_float('subsample', 0.6, 0.95),
        'task_type': 'GPU',
        'devices': '0',
        'verbose': 0,
        'allow_writing_files': False,
        'random_state': 42,
        'eval_metric': 'RMSE'
    }
    
    model = CatBoostRegressor(**params)
    
    model.fit(
        X_opt_train, y_opt_train,
        eval_set=[(X_opt_val, y_opt_val)],
        early_stopping_rounds=100
    )
    
    preds = model.predict(X_opt_val)
    return np.sqrt(mean_squared_error(y_opt_val, preds))

print("   -> Optimizing CatBoost...")
study_cat = optuna.create_study(direction='minimize')
study_cat.optimize(objective_cat, n_trials=N_TRIALS)

# --- C. LightGBM Optimization (New Callback API) ---
def objective_lgbm(trial):
    params = {
        'n_estimators': 4000,
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 30, 200),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 0.95),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 0.95),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 10),
        'device': 'gpu', # Assicurati di avere LGBM compilato per GPU, altrimenti usa 'cpu'
        'n_jobs': 1,
        'verbosity': -1,
        'random_state': 42,
        'metric': 'rmse'
    }
    
    model = LGBMRegressor(**params)
    
    # NEW API: Usiamo callbacks invece di early_stopping_rounds in .fit()
    callbacks = [
        early_stopping(stopping_rounds=100, verbose=False),
        log_evaluation(period=0) # Zittisce l'output
    ]
    
    try:
        model.fit(
            X_opt_train, y_opt_train,
            eval_set=[(X_opt_val, y_opt_val)],
            eval_metric='rmse',
            callbacks=callbacks
        )
    except Exception as e:
        # Fallback CPU se GPU crasha o non presente
        params['device'] = 'cpu'
        model = LGBMRegressor(**params)
        model.fit(
            X_opt_train, y_opt_train,
            eval_set=[(X_opt_val, y_opt_val)],
            eval_metric='rmse',
            callbacks=callbacks
        )

    preds = model.predict(X_opt_val)
    return np.sqrt(mean_squared_error(y_opt_val, preds))

print("   -> Optimizing LightGBM...")
study_lgbm = optuna.create_study(direction='minimize')
study_lgbm.optimize(objective_lgbm, n_trials=N_TRIALS)

# Recupero i migliori parametri
best_xgb_params = study_xgb.best_params
best_xgb_params.update({'n_estimators': 2000, 'tree_method': 'hist', 'device': 'gpu', 'n_jobs': 1, 'random_state': 42})
# Rimuoviamo early_stopping_rounds dai params per il fit finale (opzionale, ma pulito)
if 'early_stopping_rounds' in best_xgb_params: del best_xgb_params['early_stopping_rounds']

best_cat_params = study_cat.best_params
best_cat_params.update({'iterations': 2000, 'task_type': 'GPU', 'devices': '0', 'verbose': 0, 'random_state': 42})

best_lgbm_params = study_lgbm.best_params
best_lgbm_params.update({'n_estimators': 2000, 'device': 'gpu', 'n_jobs': 1, 'verbosity': -1, 'random_state': 42})

print("\n--- Optuna Completato. Parametri migliori trovati. ---")

# ---------------------------------------------------------
# 5. SMART SOFT MODEL SELECTOR (Mixture of Experts)
# ---------------------------------------------------------
print("\n5. Costruzione Smart Soft Selector (Mixture of Experts)...")

class SmartSoftModelSelector(BaseEstimator, RegressorMixin):
    def __init__(self, estimators, selector_model=None):
        self.estimators = estimators # Lista di tuple ('nome', modello)
        # Usiamo XGBClassifier su GPU come selettore per massima velocità
        self.selector_model = selector_model if selector_model else XGBClassifier(
            n_estimators=200, max_depth=6, learning_rate=0.05,
            tree_method='hist', device='gpu', n_jobs=1, random_state=42
        )
        self.model_names = [name for name, _ in estimators]
        
    def fit(self, X, y):
        # A. Addestramento Modelli Base
        print("   -> Training modelli base (Esperti) su tutto il Train Set...")
        self.fitted_estimators_ = []
        for name, model in self.estimators:
            # Nota: Qui non usiamo Early Stopping perché vorremmo usare tutto il train set.
            # Usiamo i parametri ottimizzati da Optuna che sono robusti.
            model.fit(X, y)
            self.fitted_estimators_.append(model)
            
        # B. Generazione OOF Predictions (Chi sbaglia meno dove?)
        print("   -> Generazione dati per il Gating Network...")
        errors = pd.DataFrame()
        
        # Nota: Usiamo n_jobs=1 in cross_val_predict perché i modelli usano GPU internamente
        for name, model in self.estimators:
            # cross_val_predict è essenziale per evitare leakage
            oof_preds = cross_val_predict(model, X, y, cv=5, n_jobs=1)
            errors[name] = np.abs(y - oof_preds) # Errore Assoluto
            
        # C. Creazione Target per il Selettore
        y_best_model_idx = errors.idxmin(axis=1).apply(lambda x: self.model_names.index(x))
        
        # D. Addestramento Selettore (Gating Network)
        print("   -> Training del Selettore (Gating Network)...")
        self.selector_model.fit(X, y_best_model_idx)
        return self

    def predict(self, X):
        # A. Predizioni Base
        base_preds = np.column_stack([model.predict(X) for model in self.fitted_estimators_])
        # B. Pesi Soft
        weights = self.selector_model.predict_proba(X)
        # C. Media Ponderata Dinamica
        final_pred = np.sum(base_preds * weights, axis=1)
        return final_pred

# Istanziamo i modelli base con i Best Params di Optuna
estimators_list = [
    ('xgb', XGBRegressor(**best_xgb_params)),
    ('cat', CatBoostRegressor(**best_cat_params)),
    ('lgbm', LGBMRegressor(**best_lgbm_params)) 
]

# Creiamo e addestriamo il sistema MoE
moe_model = SmartSoftModelSelector(estimators=estimators_list)
moe_model.fit(X_train, y_train)

# ---------------------------------------------------------
# 6. AGGIUNTA DEL CORRETTORE DEI RESIDUI (Residual Learning)
# ---------------------------------------------------------
print("\n6. Training Correttore dei Residui...")

class ResidualCorrectedMoE(BaseEstimator, RegressorMixin):
    def __init__(self, base_moe_model, corrector_model=None):
        self.base_moe_model = base_moe_model
        # Usiamo un CatBoost leggero come correttore
        self.corrector_model = corrector_model if corrector_model else CatBoostRegressor(
            iterations=500,          
            depth=6, 
            learning_rate=0.03,
            l2_leaf_reg=5,           
            task_type='GPU', 
            devices='0',
            verbose=0,
            allow_writing_files=False,
            random_state=42
        )
        
    def fit(self, X, y):
        
        # 2. Generiamo le predizioni OOF per calcolare i residui onesti
        print("   -> Calcolo residui OOF (può richiedere tempo)...")
        oof_preds = cross_val_predict(self.base_moe_model, X, y, cv=5, n_jobs=1)
        
        # 3. Calcolo dell'Errore
        residuals = y - oof_preds
        
        print(f"      Media Residui: {residuals.mean():.4f}")
        print(f"      Deviaz. Std Residui: {residuals.std():.4f}")
        
        # 4. Addestriamo il Correttore
        print("   -> Training del Correttore...")
        self.corrector_model.fit(X, residuals)
        return self

    def predict(self, X):
        base_pred = self.base_moe_model.predict(X)
        correction = self.corrector_model.predict(X)
        return base_pred + (1.0 * correction)

# Costruzione Finale
# Nota: passiamo il moe_model già addestrato, ma la classe ResidualCorrectedMoE
# userà cross_val_predict che internamente farà cloni e fit su fold.
final_system = ResidualCorrectedMoE(base_moe_model=moe_model)
final_system.fit(X_train, y_train)

# Valutazione
y_pred_corrected = final_system.predict(X_test)

final_rmse_corr = np.sqrt(mean_squared_error(y_test, y_pred_corrected))
final_r2_corr = r2_score(y_test, y_pred_corrected)

print(f"\n==========================================")
print(f" RISULTATI FINALI OTTIMIZZATI")
print(f"==========================================")
print(f" RMSE: {final_rmse_corr:.5f}")
print(f" R^2 : {final_r2_corr:.5f}")
print(f"==========================================")

# Plot veloce
corrections_test = final_system.corrector_model.predict(X_test)
base_preds_test = final_system.base_moe_model.predict(X_test)
plt.figure(figsize=(10, 6))
sns.scatterplot(x=base_preds_test, y=corrections_test, alpha=0.3)
plt.axhline(0, color='red', linestyle='--')
plt.title("Bias Correction vs Price (Optimized Models)")
plt.xlabel("Predicted Price")
plt.ylabel("Correction")
plt.show()

1. Caricamento e Pulizia Dati...
   -> Dataset pulito: (1825, 9)
2. Generazione Feature 'Combo'...
   -> Totale Feature Generate: 105
3. Selezione Feature tramite GPU...
   -> Feature Sopravvissute: 38

4. Tuning Iperparametri con Optuna e Early Stopping...
   -> Optimizing XGBoost...


[W 2025-12-10 12:19:16,126] Trial 0 failed with parameters: {'learning_rate': 0.09467957867217243, 'depth': 6, 'l2_leaf_reg': 5.153561142488177, 'subsample': 0.6809025927081477} because of the following error: CatBoostError("catboost/cuda/cuda_lib/cuda_manager.cpp:201: Condition violated: `State == nullptr'").
Traceback (most recent call last):
  File "c:\Users\m-rog\anaconda3\Lib\site-packages\optuna\study\_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\m-rog\AppData\Local\Temp\ipykernel_7292\1660013717.py", line 194, in objective_cat
    model.fit(
    ~~~~~~~~~^
        X_opt_train, y_opt_train,
        ^^^^^^^^^^^^^^^^^^^^^^^^^
        eval_set=[(X_opt_val, y_opt_val)],
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        early_stopping_rounds=100
        ^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\Users\m-rog\anaconda3\Lib\site-packages\catboost\core.py", line 5873, in fit
    return self._fit(X, y, cat_features, text_features, embeddin

   -> Optimizing CatBoost...


CatBoostError: catboost/cuda/cuda_lib/cuda_manager.cpp:201: Condition violated: `State == nullptr'